In [1]:
import pandas as pd
import numpy as np
import sys
import pickle

In [3]:
from pitched_recommend import Recommender

In [4]:
playlists = pd.read_csv('/Users/daria/Downloads/data_browse_playlists.csv')
playlists.head()

,index,territory,category_id,Position,playlist_uri,playlist_name_x,listeners,country_owner,count_appearance,mean_listeners_cat_terr,same_as_country,local,popular,score
0,57596,UY,workout,1,spotify:playlist:37i9dQZF1DX9ZKyQHcEFXZ,Para Entrenar,NaN,NaN,17,NaN,0,1,0,1
1,56668,UY,workout,10,spotify:playlist:37i9dQZF1DX4eRPd9frC1m,Hype,NaN,NaN,58,NaN,0,0,0,0
2,56727,UY,workout,12,spotify:playlist:37i9dQZF1DWSJHnPb1f0X3,Cardio,NaN,NaN,58,NaN,0,0,0,0
3,56787,UY,workout,8,spotify:playlist:37i9dQZF1DX70RN3TfWWJh,Workout,NaN,NaN,57,NaN,0,0,0,0
4,56872,UY,workout,18,spotify:playlist:37i9dQZF1DX35oM5SPECmN,Run Wild,NaN,NaN,58,NaN,0,0,0,0


In [13]:
with open('/Users/daria/Downloads/recommender_model_model_latest.pkl', 'r+b') as pickle_file:
    model = pickle.load(pickle_file)

OSError: [Errno 22] Invalid argument

In [14]:
class MacOSFile(object):
    def __init__(self, f):
        self.f = f

    def __getattr__(self, item):
        return getattr(self.f, item)

    def read(self, n):
        if n >= (1 << 31):
            buffer = bytearray(n)
            pos = 0
            while pos < n:
                size = min(n - pos, 1 << 31 - 1)
                chunk = self.f.read(size)
                buffer[pos:pos + size] = chunk
                pos += size
            return buffer
        return self.f.read(n)

In [15]:
with open("/Users/daria/Downloads/recommender_model_model_latest.pkl", "rb") as fin:
    model = pickle.load(MacOSFile(fin))

In [7]:
with open('/Users/daria/Downloads/metadata_dict.pkl', 'rb') as pickle_file:
    metadata = pickle.load(pickle_file)

In [8]:
all_territories = list(playlists.territory.unique())

In [9]:
rec = []

In [10]:
df=playlists.copy()

In [16]:
for territory in all_territories:
    #print("Producing recommendations for ", territory)
        terr_cat = list(df[df.territory==territory].category_id.unique())
        for category in terr_cat:
        #print("Producing recommendations for category ", category)
            terr_cat_playlists = list(df[(df.territory==territory)&(df.category_id==category)].playlist_uri.unique())
            rec.extend(model.rec_multiple_playlists(terr_cat_playlists, model.playlists_sparse, territory=territory,
                                          category = category, N=150))

recommendations = [(model.isrcs[x[0]],float(x[1]),x[2],x[3],x[4]) for x in rec]
recs_final = [(x[0],x[1],metadata['track_artist'][x[0]], metadata['track_title'][x[0]], metadata['major_label'][x[0]], metadata['label_studio'][x[0]],
               metadata['original_release_date'][x[0]], metadata['genre_name'][x[0]], metadata['parent_genre_name'][x[0]], 
               x[2], x[3], x[4]) for x in list(recommendations)]

all_columns = ['isrc','score','artist','title','major_label','label_studio','original_release_date','genre_name',
               'parent_genre_name','seed_playlist_uri','territory','category']
rec_df = pd.DataFrame(recs_final, columns = all_columns)
rec_df_umg = rec_df[rec_df.major_label=='UMG']

final_columns = ['territory', 'category', 'score', 'isrc', 'seed_playlist_uri', 'artist', 'title', 'original_release_date']
rec_df_final = rec_df_umg[final_columns]

rec_df_final.to_csv('/Users/daria/Downloads/manual_recommendations_20190408.csv', index=False, sep='\t')

In [19]:
all_columns = ['isrc','score','artist','title','major_label','label_studio','original_release_date','genre_name',
               'parent_genre_name','seed_playlist_uri','territory','category']
rec_df = pd.DataFrame(recs_final, columns = all_columns)
rec_df_umg = rec_df[rec_df.major_label=='UMG']

final_columns = ['territory', 'category', 'score', 'isrc', 'seed_playlist_uri', 'artist', 'title', 'original_release_date']
rec_df_final = rec_df_umg[final_columns]

rec_df_final.to_csv('/Users/daria/Downloads/manual_recommendations_20190408.csv', index=False, sep='\t')

In [23]:
rec_df_final[rec_df_final.title=='Unknown']

,territory,category,score,isrc,seed_playlist_uri,artist,title,original_release_date


In [26]:
playlists[(playlists.territory=='DE')&(playlists.category_id=='rock')].playlist_uri.unique()

array(['spotify:playlist:37i9dQZF1DX4vth7idTQch',
       'spotify:playlist:37i9dQZF1DX7fE85waV8CI',
       'spotify:playlist:37i9dQZF1DWZJhOVGWqUKF',
       'spotify:playlist:37i9dQZF1DXbz5uvvU2C9n',
       'spotify:playlist:37i9dQZF1DX1X7WV84927n',
       'spotify:playlist:37i9dQZF1DWWGFQLoP9qlv',
       'spotify:playlist:37i9dQZF1DWT2jS7NwYPVI',
       'spotify:playlist:37i9dQZF1DWWJOmJ7nRx0C',
       'spotify:playlist:37i9dQZF1DXcF6B6QPhFDv',
       'spotify:playlist:37i9dQZF1DX6xOPeSOGone'], dtype=object)